In [170]:
import pandas as pd
import numpy as np
from semopy import ModelMeans, Model, ModelEffects, gather_statistics, Optimizer, report, calc_stats
import graphviz

<h1 style="text-align:center">Dataset Loading</h1>

In [2]:
def read_data():
    df = pd.read_csv("../input/telomere_health.csv")
    df = df.drop(
        columns=[
            "socioeconomic_status",
            "bp",
            "bmi_category",
            "hr_category",
            "rr_category",
            "health_condition",
            "education_cohort",
            "marital_status",
        ]
    )
    return df

<h1 style="text-align:center">Data Preprocessing</h1>

In [3]:
def categorical_to_numeric(df, column, mapping, regex=False):
    df[column] = df[column].replace(mapping, regex=regex)


def fill_na(df, column, default):
    fill_value = np.nan
    if default == "median":
        fill_value = df[column].median()
    elif default == "mode":
        fill_value = df[column].mode()[0]

    df[column] = df[column].fillna(fill_value)

In [138]:
def preprocess_data(df):

    categorical_to_numeric(
        df,
        "cigarette_smoking",
        {
            "No information": np.nan,
            "Former Smoker": np.nan,
            "Never Smoker": 0,
            "Occasional Smoker": 1,
            "Regular Smoker": 2,
        },
    )

    categorical_to_numeric(
        df,
        "physical_activity_cohort",
        {
            "No information": np.nan,
            "Sedentary (Inactive)": 0,
            "Minimally Active": 1,
            "Lightly Active": 2,
            "Moderately Active": 3,
            "Highly Active": 4,
        },
    )

    categorical_to_numeric(
        df,
        "alcohol_drinking",
        {
            "No information": np.nan,
            "Former Drinker": np.nan,
            "Never Drinker": 0,
            "Occasional Drinker": 1,
            "Moderate Drinker": 2,
            "Heavy Drinker": 3,
        },
    )

    categorical_to_numeric(
        df,
        "bp_category",
        {
            "No information": np.nan,
            "Hypotension (Low BP)": 0,
            "Normal BP": 1,
            "Elevated BP": 2,
            "Hypertension Stage 1": 3,
            "Hypertension Stage 2": 4,
            "Hypertensive Crisis": 5,
        },
    )

    categorical_to_numeric(
        df,
        "cardiovascular_disease_diagnosis",
        {
            "^No known*": 0,
            "^Non-cardiovascular*": 0,
            "^Single.*": 1,
            "^Multi.*": 1,
        },
        regex=True,
    )

    categorical_to_numeric(
        df,
        "cancer_diagnosis",
        {
            "^Clinically Healthy*": 0,
            "^Non-oncologic*": 0,
            "^Single.*": 1,
            "^Multi.*": 1,
        },
        regex=True,
    )

    categorical_to_numeric(
        df,
        "depression_anxiety_diagnosis",
        {
            "^No Known*": 0,
            "^Anxiety and/or*": 1,
            "^Multi.*": 1,
        },
        regex=True,
    )

    categorical_to_numeric(
        df,
        "allergy_diagnosis",
        {
            "^No Diagnosed*": 0,
            "^Single.*": 1,
            "^Multi.*": 1,
        },
        regex=True,
    )

    categorical_to_numeric(
        df,
        "diabetes_diagnosis",
        {
            "^No known diagnosis*": 0,
            "^Diagnosed with*": 1,
        },
        regex=True,
    )

    categorical_to_numeric(
        df,
        "sex",
        {
            "Female": 0,
            "Male": 1,
        },
        regex=True,
    )

    df["hr"] = pd.to_numeric(df["hr"], errors="coerce")
    df["rr"] = pd.to_numeric(df["rr"], errors="coerce")

    fill_na(df, "hr", "median")
    fill_na(df, "rr", "median")
    fill_na(df, "bmi", "median")
    fill_na(df, "alcohol_drinking", "mode")
    fill_na(df, "cigarette_smoking", "mode")
    fill_na(df, "bp_category", "mode")
    fill_na(df, "physical_activity_cohort", "mode")

    return df

In [139]:
raw_data = read_data()
data = preprocess_data(raw_data)

C:\Users\joshua\AppData\Local\Temp\ipykernel_7800\1943607600.py:2: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df[column] = df[column].replace(mapping, regex=regex)


In [140]:
data

,telomere_length,age,sex,cigarette_smoking,alcohol_drinking,physical_activity_cohort,bmi,hr,rr,bp_category,cardiovascular_disease_diagnosis,cancer_diagnosis,depression_anxiety_diagnosis,allergy_diagnosis,diabetes_diagnosis
0,11.34,18,0,0.0,1.0,3.0,21.400000,75.0,21.0,1.0,0,0,0,0,0
1,22.15,21,0,0.0,1.0,2.0,22.700000,75.0,20.0,1.0,0,0,0,0,0
2,1.81,20,0,0.0,0.0,3.0,24.200000,77.0,21.0,3.0,0,0,0,0,0
3,8.97,23,0,0.0,2.0,4.0,16.000000,60.0,21.0,1.0,0,0,0,0,0
4,8.36,18,1,0.0,2.0,4.0,29.800000,77.0,18.0,3.0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
610,7.48,36,1,0.0,2.0,1.0,23.329442,77.0,18.0,3.0,0,0,1,0,0
611,4.91,24,1,0.0,2.0,3.0,23.329442,77.0,18.0,3.0,0,0,1,0,0
612,12.81,37,1,0.0,2.0,0.0,23.329442,77.0,18.0,3.0,0,0,1,0,0
613,7.06,19,1,0.0,0.0,3.0,23.329442,77.0,18.0,3.0,0,0,1,0,0


<h1 style="text-align:center">Structural Equation Modelling</h1>

In [141]:
columns = data.columns
col_list = " + ".join(columns)

In [ ]:
desc = """
telomere_length ~ age + cigarette_smoking + alcohol_drinking + physical_activity_cohort + bmi + hr + rr + bp_category
"""



model = Model(desc)

model.fit(data, obj="WLS")

SolverResult(fun=np.float64(7.077424089936488e-09), success=np.True_, n_it=42, x=array([ 3.47684428e-02, -1.55893817e-01,  3.14203682e-01, -1.01983723e+00,
       -1.14940580e-02, -4.09597879e-02, -2.11204452e-01,  3.48823110e-01,
        6.48500706e+01]), message='Optimization terminated successfully', name_method='SLSQP', name_obj='WLS')

In [160]:
model_result = model.inspect()
model_result["sig"] = model_result["p-value"].apply(lambda p: "*" if p < 0.05 else "")

In [161]:
model_result

,lval,op,rval,Estimate,Std. Err,z-value,p-value,sig
0,telomere_length,~,age,0.034768,0.019143,1.816220,0.069337,
1,telomere_length,~,cigarette_smoking,-0.155894,0.516934,-0.301574,0.762977,
2,telomere_length,~,alcohol_drinking,0.314204,0.415693,0.755854,0.449737,
3,telomere_length,~,physical_activity_cohort,-1.019837,0.248164,-4.109535,0.000040,*
4,telomere_length,~,bmi,-0.011494,0.018613,-0.617540,0.536878,
5,telomere_length,~,hr,-0.040960,0.032082,-1.276706,0.201706,
6,telomere_length,~,rr,-0.211204,0.175356,-1.204432,0.228423,
7,telomere_length,~,bp_category,0.348823,0.344436,1.012736,0.311186,
8,telomere_length,~~,telomere_length,64.850071,3.698179,17.535678,0.000000,*


In [171]:
model_fit = calc_stats(model)

In [172]:
model_fit

,DoF,DoF Baseline,chi2,chi2 p-value,chi2 Baseline,CFI,GFI,AGFI,NFI,TLI,RMSEA,AIC,BIC,LogLik
Value,36,44,0.000004,1.0,466.391259,1.085229,1.0,1.0,1.0,1.104169,0,18.0,57.7946,1.378562e-08


In [166]:
report(model, "Telomere Health")